In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)

train_dir = "output/train/"
test_dir = "output/val/"



train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                                label_mode="categorical",
                                                                                image_size=IMG_SIZE)
                                                                                
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE,
                                                                shuffle=False)



Found 4764 files belonging to 80 classes.
Found 1602 files belonging to 80 classes.


In [9]:
print(train_data)

<BatchDataset shapes: ((None, 224, 224, 3), (None, 80)), types: (tf.float32, tf.float32)>


In [10]:
checkpoint_path = "better_model_checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True, # save only the model weights
                                                         monitor="val_accuracy", # save the model weights which score the best validation accuracy
                                                         save_best_only=True)

In [11]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

# original_data = Sequential([
#   preprocessing.RandomRotation(0), 
# ], name="original_data")

data_augmentation = Sequential([
    preprocessing.RandomRotation(0),
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomRotation(0.2),  
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomZoom(0.2),
    preprocessing.RandomContrast(0.1),
], name="data_augmentation")

In [12]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False


inputs = layers.Input(shape=(224, 224, 3), name="input_layer")

x = base_model(inputs, training=False) 
x = layers.GlobalAveragePooling2D(name="global_average_pooling")(x) 
outputs = layers.Dense(len(train_data.class_names), activation="softmax", name="output_layer")(x) 
model = tf.keras.Model(inputs, outputs)

In [13]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
global_average_pooling (Glob (None, 1280)              0         
_________________________________________________________________
output_layer (Dense)         (None, 80)                102480    
Total params: 4,152,051
Trainable params: 102,480
Non-trainable params: 4,049,571
_________________________________________________________________


In [14]:
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

# Fit
history_base_model = model.fit(train_data,
                                           epochs=10, 
                                           validation_data=test_data,
                                           validation_steps=len(test_data))

Epoch 1/10
149/149 [==============================] - 55s 350ms/step - loss: 3.5037 - accuracy: 0.2974 - val_loss: 2.6613 - val_accuracy: 0.6092
Epoch 2/10
149/149 [==============================] - 53s 356ms/step - loss: 2.1345 - accuracy: 0.7238 - val_loss: 1.7720 - val_accuracy: 0.8034
Epoch 3/10
149/149 [==============================] - 50s 337ms/step - loss: 1.4144 - accuracy: 0.8707 - val_loss: 1.2713 - val_accuracy: 0.8783
Epoch 4/10
149/149 [==============================] - 52s 346ms/step - loss: 1.0033 - accuracy: 0.9217 - val_loss: 0.9698 - val_accuracy: 0.9095
Epoch 5/10
149/149 [==============================] - 51s 345ms/step - loss: 0.7464 - accuracy: 0.9513 - val_loss: 0.7774 - val_accuracy: 0.9251
Epoch 6/10
149/149 [==============================] - 52s 347ms/step - loss: 0.5829 - accuracy: 0.9652 - val_loss: 0.6369 - val_accuracy: 0.9419
Epoch 7/10
149/149 [==============================] - 51s 344ms/step - loss: 0.4668 - accuracy: 0.9746 - val_loss: 0.5432 - val_ac

In [ ]:
base_model.trainable = True

# Refreeze every layer except for the last 5
for layer in base_model.layers[:-5]:
    layer.trainable = False


In [ ]:
new_model = model

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4), # 10x lower learning rate than default
              metrics=['accuracy'])

In [ ]:
fine_tune_epochs = 31

history_model_fine_tune = model.fit(train_data,
                                                     epochs=fine_tune_epochs,
                                                     validation_data=test_data,
                                                     validation_steps=len(test_data),
                                                     initial_epoch=history_base_model.epoch[-1])

In [ ]:
# import cv2
# import numpy as np
# IMAGE_SHAPE = (224, 224)
# IMAGE_SHAPE+(3,)
# test_dir = "C:\\Users\\bhave\\Project-1\\Class25_Dataset\\train\\a\\2.jpeg"
# img = cv2.imread(test_dir)
# img_resize = cv2.resize(img , IMAGE_SHAPE)
# predicted = model.predict(np.array([img_resize]))
# ind = np.argmax(predicted , axis = 1)
# index = ind[0]
# print(index)
# cv2.imshow('detected', img)
# cv2.waitKey(0)

In [15]:
model_json = model.to_json()
with open("2ndtry_better_model.json" , "w") as json_file:
    json_file.write(model_json)
model.save_weights("2ndtry_better_model_weights.h5")

/usr/local/lib/python3.6/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
from keras.models import model_from_json
import json

model_file = open("better_model.json", "r")
model_json = model_file.read()
model = model_from_json(model_json)
model.load_weights("better_model_weights.h5")